In [2]:
!pip install transformers gdown --quiet


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# Примеры
input_texts = [
    "Когда был спущен на воду первый миноносец «Спокойный»?",
    "Есть ли нефть в Удмуртии?",
    "Спокойный (эсминец)\nЗачислен в списки ВМФ СССР 19 августа 1952 года.",
    "Нефтепоисковые работы в Удмуртии были начаты сразу после Второй мировой войны в 1945 году и продолжаются по сей день. Добыча нефти началась в 1967 году."
]

device = "cuda" if torch.cuda.is_available() else "cpu"

# Загружаем модель и токенизатор
tokenizer = AutoTokenizer.from_pretrained("deepvk/USER-bge-m3", use_fast=True)
model = AutoModel.from_pretrained("deepvk/USER-bge-m3")
model.to(device)
model.eval()

# Включаем TF32 для всех float32 матричных умножений
torch.set_float32_matmul_precision('high')

# Опционально ускоряем на через compile
if hasattr(torch, 'compile'):
    model = torch.compile(model)


def model_encode(input_texts, batch_size = 64):
    all_embeddings = []

    for i in tqdm(range(0, len(input_texts), batch_size), desc="Encoding batches"):
        batch_texts = input_texts[i:i+batch_size]
        
        encoded_input = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            return_tensors='pt'
        ).to(device)
        
        attention_mask = encoded_input['attention_mask']

        with torch.no_grad():
            model_output = model(**encoded_input)
            token_embeddings = model_output[0]  # [batch, seq_len, hidden]

            # Mean pooling и маска внимания
            mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            summed = torch.sum(token_embeddings * mask, dim=1)
            summed_mask = torch.clamp(mask.sum(dim=1), min=1e-9)
            embeddings = summed / summed_mask

            # Нормализация
            embeddings = nn.functional.normalize(embeddings, p=2, dim=1)
            all_embeddings.append(embeddings.cpu())

    # Объединяем все батчи
    sentence_embeddings = torch.cat(all_embeddings, dim=0)
    return sentence_embeddings


print(model_encode(input_texts).shape)


Encoding batches: 100%|██████████| 1/1 [00:13<00:00, 13.91s/it]

torch.Size([4, 1024])


In [2]:
import json

with open("chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)


In [ ]:
# Чтобы не запускать эмбеддер, можно скачать готовые эмебеддинги и чанки

!gdown 10PIu9zXbrKVRfP_hGDJDgxdM7PXN6lmD  
!gdown 17pSAjmKKcaMbMr1TnpS9Qf0NxRPzOsCy 

Downloading...
From: https://drive.google.com/uc?id=10PIu9zXbrKVRfP_hGDJDgxdM7PXN6lmD
To: /teamspace/studios/this_studio/chunks_sub.json
100%|██████████████████████████████████████| 46.9M/46.9M [00:01<00:00, 39.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=17pSAjmKKcaMbMr1TnpS9Qf0NxRPzOsCy
From (redirected): https://drive.google.com/uc?id=17pSAjmKKcaMbMr1TnpS9Qf0NxRPzOsCy&confirm=t&uuid=3e1ccc58-db8f-45cb-9825-61eb4a2f0f87
To: /teamspace/studios/this_studio/embeddings.pt
100%|██████████████████████████████████████| 81.9M/81.9M [00:02<00:00, 32.8MB/s]


In [3]:
import random 

# Для теста уменьшенный возьмем датасет 
subsample = random.choices(chunks, k = 20000)

In [4]:
embs = model_encode(subsample, batch_size = 192) 


Encoding batches: 100%|██████████| 105/105 [09:52<00:00,  5.64s/it]


In [6]:
torch.save(embs, "embeddings.pt") 

In [7]:
import json 

with open("chunks_sub.json", "w", encoding="utf-8") as f:
    json.dump(subsample, f, ensure_ascii=False, indent=2)